In [ ]:
from underlying import Underlying
from option import Option, Vanilla
from trader import Trader

import matplotlib.pyplot as plt
from tqdm import tqdm
from quantile_hedging_calculator import *
import numpy as np
import pandas as pd

In [ ]:
def payoff_call(X, K):
    return np.maximum(X.iloc[:,-1]- K, 0)
def payoff_put(X, K):
    return np.maximum(K - X.iloc[:,-1], 0)

In [ ]:
mu = 0.01
sigma =  0.11
r = 0.05
T = 1
X0 = 100
repeat = 100

## Full hedging - Vanilla Call

In [ ]:
K = 80

In [ ]:
underlying = Underlying(mu, sigma, r, 250)

In [ ]:
_, reality = underlying.simulate_P(repeat, T)

In [ ]:
vanilla_call = Option(underlying, lambda X: payoff_call(X, K), T)

In [ ]:
BS_price_call = vanilla_call.get_MC_delta(X0, 0)
BS_price_call

In [ ]:
call_vanilla = Vanilla(underlying, K, T, True)
print(call_vanilla.get_delta(X0))

In [ ]:
money_time_call = pd.DataFrame(np.zeros(reality.shape))
delta_time_call = pd.DataFrame(np.zeros(reality.shape))
for i in tqdm(range(repeat)):
    trader = Trader(initial_capital = BS_price_call)
    money, delta, _ = trader.simulate_hedging(vanilla_call, X0*reality.iloc[[i],:], update_freq = 1)
    money_time_call.loc[i] = money
    delta_time_call.loc[i] = delta

In [ ]:
(X0*reality).T.plot(legend = False)

In [ ]:
(money_time_call).T.plot(legend = False)

In [ ]:
(delta_time_call).T.plot(legend = False)

In [ ]:
plt.hist(money_time_call.iloc[:,-1])

## Full hedging - Vanilla Put

In [ ]:
K = 120

In [ ]:
underlying = Underlying(mu, sigma, r, 250)

In [ ]:
_, reality = underlying.simulate_P(repeat, T)

In [ ]:
vanilla_put = Option(underlying, lambda X: payoff_put(X, K), T)

In [ ]:
BS_price_put = vanilla_put.get_MC_price(X0, 0)
BS_price_put

In [ ]:
money_time_put = pd.DataFrame(np.zeros(reality.shape))
delta_time_put = pd.DataFrame(np.zeros(reality.shape))
for i in tqdm(range(repeat)):
    trader = Trader(initial_capital = BS_price_put)
    money, delta, _ = trader.simulate_hedging(vanilla_put, X0*reality.iloc[[i],:], update_freq = 1)
    money_time_put.loc[i] = money
    delta_time_put.loc[i] = delta

In [ ]:
(X0*reality).T.plot(legend = False)

In [ ]:
(money_time_put).T.plot(legend = False)

In [ ]:
(delta_time_put).T.plot(legend = False)

In [ ]:
plt.hist(money_time_put.iloc[:,-1])

## Quantile hedging with half of required initial capital- Vanilla Call

In [ ]:
K = 80

In [ ]:
underlying = Underlying(mu, sigma, r, 250)

In [ ]:
_, reality = underlying.simulate_P(repeat, T)

In [ ]:
money_time_call = pd.DataFrame(np.zeros(reality.shape))
delta_time_call = pd.DataFrame(np.zeros(reality.shape))
for i in tqdm(range(repeat)):
    trader = Trader(initial_capital = BS_price_call/2)
    money, delta, success_prob = trader.simulate_hedging(vanilla_call, X0*reality.iloc[[i],:], update_freq = 1, limited_capital = True)
    money_time_call.loc[i] = money
    delta_time_call.loc[i] = delta
print(f'Success probability with initial capital={BS_price_call/2:.2f} is: {success_prob}')

In [ ]:
(X0*reality).T.plot(legend = False)

In [ ]:
(money_time_call).T.plot(legend = False)

In [ ]:
(delta_time_call).T.plot(legend = False)

In [ ]:
plt.hist(money_time_call.iloc[:,-1])

In [ ]:
print(f'Empiric success probability with initial capital={BS_price_call/2:.2f} is: {(money_time_call.iloc[:,-1] >= -0.1*BS_price_call/2).mean()}')

## Quantile hedging with half of required initial capital- Vanilla Put

In [ ]:
K = 120

In [ ]:
underlying = Underlying(mu, sigma, r, 250)

In [ ]:
_, reality = underlying.simulate_P(repeat, T)

In [ ]:
money_time_put = pd.DataFrame(np.zeros(reality.shape))
delta_time_put = pd.DataFrame(np.zeros(reality.shape))
for i in tqdm(range(repeat)):
    trader = Trader(initial_capital = BS_price_put/2)
    money, delta, success_prob = trader.simulate_hedging(vanilla_put, X0*reality.iloc[[i],:], update_freq = 1, limited_capital = True)
    money_time_put.loc[i] = money
    delta_time_put.loc[i] = delta
print(f'Success probability with initial capital={BS_price_put/2:.2f} is: {success_prob}')

In [ ]:
(X0*reality).T.plot(legend = False)

In [ ]:
(money_time_put).T.plot(legend = False)

In [ ]:
(delta_time_put).T.plot(legend = False)

In [ ]:
plt.hist(money_time_put.iloc[:,-1])

In [ ]:
print(f'Empiric success probability with initial capital={BS_price_put/2:.2f} is: {(money_time_put.iloc[:,-1] >= -0.1*BS_price_put/2).mean()}')